In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sb
from scipy.stats import pareto
from scipy.stats import norm
from random import uniform
from sympy.abc import x
from sympy import Poly
from sympy.solvers.inequalities import solve_rational_inequalities
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.collections import PolyCollection   # these are bunch of header files some aare needed some may be required for future use
import random
import pandas as pd

np.random.seed(0)
random.seed(0)

%matplotlib inline

In [ ]:
population_size = 100000
thetas = (1.15, 1.15, 1.7)
quantiles = np.linspace(0.00001,0.99999, population_size)

Z0 = pareto.ppf(quantiles, thetas[0])
Z1 = pareto.ppf(quantiles, thetas[1])
Z2 = pareto.ppf(quantiles, thetas[2])


fig, axs = plt.subplots(1,3, figsize = (25,5))

pd.Series(Z0).plot.kde(ax = axs[0], xlim = (0,1000))
pd.Series(Z1).plot.kde(ax = axs[1], xlim = (0,1000))
pd.Series(Z2).plot.kde(ax = axs[2], xlim = (0,1000))

axs[0].set_title('Distribution for Theta {}'.format(thetas[0]))
axs[1].set_title('Distribution for Theta {}'.format(thetas[1]))
axs[2].set_title('Distribution for Theta {}'.format(thetas[2]))

In [ ]:
W0, W1, W2 = Z0, Z1, Z2*5

A_high = 1
A_low = 0.9


Wage_high = A_high * W0
Wage_low = A_low * W0


discount_beta, return_R = 0.9, 1.05

In [ ]:
# BASED ON EQUATION XXX of PAPER
def calculate_savings(probability_wage_low, Wage_high, Wage_low):
    W_hat = probability_wage_low*Wage_high + (1-probability_wage_low)*Wage_low
    W_bar = Wage_high * Wage_low
    omega = Wage_low + Wage_high   + discount_beta *(W_hat - (W0* return_R))
    
    denominator = 2* (1+ discount_beta) * return_R
    numerator = -omega +np.sqrt((omega**2)-(4*(1+discount_beta)*W_bar) + (4 * discount_beta * (1+discount_beta) * return_R * W0 * W_hat ))
    return numerator/denominator


In [ ]:
# BASED ON EQUATION XXX of PAPER
α = .6
def decision_function(ρ, Wage_low , savings , probability_wage_low , Wage_high ,migration_cost ):
    sub_1 = α**(-ρ)
    sub_2 = (Wage_low + return_R*savings)**(probability_wage_low )
    sub_3 = (Wage_high + return_R*savings)**(1-probability_wage_low )
    sub_4 = W0-savings
    sub_5 = W0-migration_cost
    sub_6 = (sub_4/sub_5)**(1/discount_beta)
    
    output = sub_1*sub_2*sub_3*sub_6
    return output 
 
def prob_downgrade(γ,Z):
    sub_1 = -3
    sub_2 = (Z+0.5)**(-0.3)
    sub_3 = 1.17
    return  (sub_1*sub_2)+sub_3                   # ((-3/(Z+0.5)**(-.3))+1.17) *γ #gamma should be defined such taht rho is between (0,1)

def expected_wages(ρ,Z,α):
    sub_1 = ρ*Z*α
    sub_2 = (1-ρ)*Z
    return sub_1+sub_2
gamma11=prob_downgrade(.8,W1)
#kk =decision_function(gamma11,0.7*W0,savings,.0.01,W0,3)
gamma11
decision_function(gamma11,0.7*W0,)

In [ ]:
def conditional_decision_function(C1, C2):
#     df, wage, color = C1, W1, '#000000'
    if np.all(np.all(W1 > W2) and np.all(C1 <= W1)):
        df = C1
        color = '#5090d4'
        wage = W1
    elif np.all(np.all(W1 <= W2) and np.all(C2 <= W2)):
        df = C2
        color = '#000000'
        wage = W2
    else:
        if np.all(W2>=W1):
            df = C2
            color = '#000000'
            wage = W2
        elif np.all(W1>W2):
            df = C1
            color = '#5090d4'
            wage = W1
        else:
            df = np.maximum(C1, C2)
            color = '#5090d4'
            wage = np.maximum(W1,W2)
            
    return df, wage, color


In [ ]:
probabilities_wage_low = [0.01,0.4, 0.8,0.8]
migration_costs = [(3,3,3,10),(7,7,7,20)]
A_lows = (0.7, 0.2, 0.2, 0.2)
gamma= .8,.85  
fig,axs = plt.subplots(4,3 ,figsize=(25,20)) 
fig.suptitle("Fixed Migration Costs", fontsize=20, y=1.05)

for idx in range(12):
    r, c = idx//3, idx%3
    
    if c in [0,1]:
        WL, WH, MC, p = A_lows[r]* W0, Wage_high, migration_costs[c][r], probabilities_wage_low[r]
        rho = prob_downgrade(gamma[0],W1) if c==0 else prob_downgrade(gamma[1],W2)
        savings = calculate_savings(probability_wage_low = p, Wage_high = WH, Wage_low = WL)
        decision_function_out = decision_function(ρ =rho, Wage_low = WL, savings = savings, probability_wage_low = p, Wage_high = WH,migration_cost = MC)
        non_migration_wage = expected_wages(ρ=rho,Z=W1,α=.6) if c ==0 else expected_wages(ρ=rho,Z=W2,α=.6)
        
        color = '#5090d4' if c == 0 else '#000000'
        axs[r,c].plot(W0, decision_function_out, label = "Indifference Curve", c = color)
        axs[r,c].plot(W0, non_migration_wage, label = "Wage Distribution", c = '#f76605')

    else:
        WL, WH, MC, p = A_lows[r]* W0, Wage_high, migration_costs[0][r], probabilities_wage_low[r]
        savings = calculate_savings(probability_wage_low = p, Wage_high = WH, Wage_low = WL)
        C1 = decision_function(ρ =rho, Wage_low = WL, savings = savings, probability_wage_low = p, Wage_high = WH,migration_cost = MC)
        
        WL, WH, MC, p = A_lows[r]* W0, Wage_high, migration_costs[1][r], probabilities_wage_low[r]
        savings = calculate_savings(probability_wage_low = p, Wage_high = WH, Wage_low = WL)
        C2 = decision_function(ρ =rho, Wage_low = WL, savings = savings, probability_wage_low = p, Wage_high = WH,migration_cost = MC)
        
        MC = [migration_costs[0][r], migration_costs[1][r]]
#         decision_function_out,wage, color = conditional_decision_function(C1, C2)
        
        axs[r,c].plot(W0, C1, label = "Indifference Curve 1", c = '#5090d4')
        axs[r,c].plot(W0, C2, label = "Indifference Curve 2", c = '#000000')
        axs[r,c].plot(W0, W1, label = "Wages Neighbour", c = '#f76605')
        axs[r,c].plot(W0, W2, label = "Wage Foreign", c = '#ff0000')
        
    txt = "P = {} \n MC = {} \n A_l ={}".format(p, MC,  A_lows[r])
    axs[r,c].text(0.98,0.0, txt, bbox={'facecolor': 'aliceblue', 'alpha': 0.2, 'pad': 1},fontsize=15, transform=axs[r,c].transAxes, horizontalalignment='right', verticalalignment='bottom')

    axs[r,c].set_xlabel('Country {}'.format(c+1))
    axs[r,c].set_xlim([0, 1000])
    axs[r,c].set_ylim([0, 1000])
    axs[r,c].legend()
    axs[r,c].set_xlabel("W 0")
    
    if  c == 0:
        axs[r,c].set_ylabel("W 1")
    elif c==1:
        axs[r,c].set_ylabel("W 2")
    else:
        axs[r,c].set_ylabel("W")
    plt.tight_layout()
